In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/", one_hot=True)

##定义超参数
batch_size = 128
learning_rate = 0.8
learning_rate_decay = 0.999
max_steps = 30000

##定义存储训练轮数的变量，在使用tensorflow训练神经网络时，一般会将代表训练轮数的变量设置为不可训练的
training_step = tf.Variable(0, trainable=False)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
##定义完成后，即可搭建神经网络前馈学习过程
def hidden_layer(input_tensor, weights1, biases1, weights2, biases2, layer_name):
    layer1 = tf.nn.relu(tf.matmul(input_tensor,weights1) + biases1)
    return tf.matmul(layer1, weights2) + biases2

x = tf.placeholder(tf.float32, [None, 784], name='x-input')
y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')

##生层隐藏层参数，其中weights包含784x500个参数
weights1 = tf.Variable(tf.truncated_normal([784, 500], stddev=0.1))
biases1 = tf.Variable(tf.constant(0.1, shape=[500]))

##生成输出层参数，其中weights包含500x10个参数

weights2 = tf.Variable(tf.truncated_normal([500, 10], stddev=0.1))
biases2 = tf.Variable(tf.constant(0.1, shape=[10]))

y = hidden_layer(x, weights1, biases1, weights2, biases2, 'y')

In [4]:
##定义滑动平均值
averages_class = tf.train.ExponentialMovingAverage(0.99, training_step)

##定义一个更新变量滑动平均值的操作需要向滑动平均累的apply提供一个参数列表
##train_variable函数返回计算图中所有需要训练的参数
averages_op = averages_class.apply(tf.trainable_variables())

##再次计算经过神经网络前向传播后得到的y值，这里使用了滑动平均，但要牢记滑动平均值只是一个影子变量
average_y = hidden_layer(x, averages_class.average(weights1), averages_class.average(biases1), averages_class.average(weights2), averages_class.average(biases2), 'average_y')

print(average_y)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Tensor("add_3:0", shape=(?, 10), dtype=float32)


In [7]:
###定义损失函数
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))

##定义正则化regularizer
regularizer = tf.contrib.layers.l2_regularizer(0.0001)

regularization = regularizer(weights1) + regularizer(weights2)  ## 计算模型的正则化损失

loss = tf.reduce_mean(cross_entropy) + regularization ##总损失

##指数衰减法设置学习率，staircase参数控制是否连续衰减
learning_rate = tf.train.exponential_decay(learning_rate, training_step, mnist.train.num_examples/batch_size, learning_rate_decay)

##定义优化器
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=training_step)


##通过control_dependencies()来完成对网络中的参数以及参数的平均值进行更新
with tf.control_dependencies([train_step, averages_op]):
    train_op = tf.no_op(name='train')

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [8]:
##检查使用了滑动平均模型的神经网络前向传播结果是否正确
crorent_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
print(y_)
accuracy = tf.reduce_mean(tf.cast(crorent_prediction, tf.float32))  ###转换类型

Tensor("y-input:0", shape=(?, 10), dtype=float32)


In [14]:
##训练
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    ##准备验证数据
    validate_feed = {x: mnist.validation.images, y_:mnist.validation.labels}
    
    ##准备测试数据
    test_feed = {x: mnist.test.images, y_:mnist.test.labels}
    
#     for i in range(2000 + 1):
#         xs, ys = mnist.train.next_batch(batch_size=batch_size)
#         print(ys.shape)
# #         ys = ys.reshape([-1, 10])
#         _, loss_value = sess.run([train_op, loss], feed_dict={x:xs, y_:ys})
#         if i % 1000:
#             ##计算滑动平均模型在验证数据上的结果
#             validate_accuracy = sess.run(accuracy, feed_dict=validate_feed)
            
#             print("after {} training step(s), validation accuracy using average model is {}% loss={}".format(i, round(validate_accuracy * 100), loss_value))
#         ##产生训练数据
        

    test_accuracy = sess.run(accuracy, feed_dict=test_feed)
    pred_y = sess.run(tf.argmax(average_y, 1), feed_dict=test_feed)
    print(pred_y)
    print("after {} training step(s), validation accuracy using average model is {}%".format(max_steps, test_accuracy * 100))


[4 0 3 ... 5 8 6]
after 30000 training step(s), validation accuracy using average model is 10.769999772310257%


In [15]:
pred_y.shape

(10000,)